In [1]:
# Notebook demonstrates end to end session with planout
# experiment library including experimental specification,
# simulation of decisions, reading of logs and analysis of data.

# The simulated experiment is the proverbial 'button colour'
# A/B test.

# References:

# https://facebook.github.io/planout/
# Bakshy et al 2014

# Extensions
# - Log outcomes.
# - Multiple parameters.
# - Multiple units (userid, contentid).
# - Exposure over time.

import math
import json
import numpy as np
import pandas as pd
from scipy.stats import bernoulli, norm
from planout.experiment import SimpleExperiment
from planout.ops.random import UniformChoice

In [2]:
# Define experiment

class ButtonExperiment(SimpleExperiment):
  def assign(self, params, userid):
    params.button_color = UniformChoice(choices=['A', 'B'], unit=userid)

In [3]:
# Simulate exposures and click throughs

num_users = 1000

# Simulated click through rates
ctr = {'A': 0.5, 'B': 0.6}

for userid in range(0, num_users):
    exp = ButtonExperiment(userid=userid)

    # Log exposure to VotingExperiment.log
    call_to_action_color = exp.get('button_color')
    
    # Log an action
    simulated_decision = bernoulli.rvs(ctr[call_to_action_color])
    if simulated_decision:
        exp.log_event('button_click')

In [4]:
# Now perform summary statistics for each unique combination of parameters

# params = A, B
# num_exposure, num_click

# Read planout logs (rows of json data)
data = []
with open(exp.name+'.log') as file:
    for line in file:
       data.append(json.loads(line))

# Extract event and params data. Convert params to string.
event = []
params = []
for line in data:
    event.append(line['event'])
    params.append(str(line['params']))
                  
# Insert data into a data frame
df = pd.DataFrame.from_items([('params', params), ('event', event) ])

# Calculate cross table to get summary statistics
crosstab = pd.crosstab(df.params, df.event)
print(crosstab)

event                  button_click  exposure
params                                       
{'button_color': 'A'}           267       519
{'button_color': 'B'}           300       481


In [5]:
# Discussion of Z statistic here:
# https://making.lyst.com/2014/05/10/bayesian-ab-testing/

def zstat(crosstab):
    '''Calculate the Z statistic for an A/B test given
    the summary statistics in a crosstab (exposures and events)'''
    
    crosstab_values = crosstab.copy().values
    nrow = crosstab_values.shape[0]

    success_rate = []
    exposure = []
    total_events = 0
    total_exposures = 0
    for row in range(0, nrow):
        events = crosstab_values[row, 0]
        exposures = crosstab_values[row, 1]
        
        rate = events/exposures
        print(crosstab.axes[0][row], 'success rate = ', rate)
        
        success_rate.append(rate)
        exposure.append(exposures)
        
        total_exposures += exposures
        total_events += events
    
    combined_success_rate = total_events/total_exposures
    
    return (success_rate[1] - success_rate[0])/ \
            math.sqrt(combined_success_rate*(1.-combined_success_rate)* \
            (1.0/exposure[0] + 1.0/exposure[1]))

    
abtest_zstat = zstat(crosstab)
print('Z statistic = ', abtest_zstat)
print('Cumulative area under normal distribution =',norm.cdf(abtest_zstat))

{'button_color': 'A'} success rate =  0.514450867052
{'button_color': 'B'} success rate =  0.623700623701
Z statistic =  3.48370376386
Cumulative area under normal distribution = 0.99975273674
